In [1]:
import os
from dotenv import load_dotenv
from google.cloud import bigquery
import pandas as pd

In [2]:
# --- Carga das variáveis de ambiente do arquivo .env ---
load_dotenv()

True

In [3]:
# --- Configuração da autenticação com Google Cloud ---
current_dir = os.getcwd()
json_path = os.path.abspath(os.path.join(current_dir, '..', 'credenciais', 'bq-chave.json'))

print(f"Caminho do arquivo de credenciais: {json_path}")
print(f"Arquivo existe? {os.path.exists(json_path)}")  # Deve ser True

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = json_path

# --- Inicializa cliente BigQuery ---
client = bigquery.Client()

# --- Definições padrão do projeto, dataset e tabela ---
project_id = 'telemedicina-462920'
dataset_id = 'seguros'

# Função para construir o full_table_id no formato 'projeto.dataset.tabela'
def get_full_table_id(project, dataset, table):
    return f"{project}.{dataset}.{table}"

Caminho do arquivo de credenciais: C:\Users\rodri\OneDrive\Sincronização\Documentos\GitHub\database-connection_dashboard_telemedicine\credenciais\bq-chave.json
Arquivo existe? True


In [4]:
# --- Função genérica para carregar dataframe Excel e enviar para BigQuery ---
def upload_to_bigquery_from_excel(sheet_name: str, table_name: str, schema: list):
    excel_path = os.getenv("EXCEL_FILE_PATH")
    if not excel_path or not os.path.exists(excel_path):
        raise FileNotFoundError("Arquivo Excel não encontrado ou variável EXCEL_FILE_PATH não definida.")
    
    # Leitura dos dados
    df = pd.read_excel(excel_path, sheet_name=sheet_name)
    print(f"\nDados carregados da aba '{sheet_name}':")
    print(f"Tamanho (linhas, colunas): {df.shape}")
    print(f"Tipos de dados:\n{df.dtypes}")
    print(f"Primeiras linhas:\n{df.head()}")
    
    # Configuração do job de carga com schema e modo escrita
    job_config = bigquery.LoadJobConfig(
        schema=schema,
        write_disposition="WRITE_TRUNCATE"  # Substitui a tabela toda
    )
    
    # Monta o ID completo da tabela
    full_table_id = get_full_table_id(project_id, dataset_id, table_name)
    
    # Envio dos dados para o BigQuery
    job = client.load_table_from_dataframe(df, full_table_id, job_config=job_config)
    job.result()  # Espera o término do job
    
    print(f"✅ Dados enviados com sucesso para: {full_table_id}")

In [5]:
# --- Exemplo de uso da função para a aba e tabela meta_odonto ---
schema_meta_odonto = [
    bigquery.SchemaField("Mes", "DATE"),
    bigquery.SchemaField("Meta", "INTEGER"),
]

upload_to_bigquery_from_excel(
    sheet_name='meta_odonto', 
    table_name='meta_odonto', 
    schema=schema_meta_odonto
)


Dados carregados da aba 'meta_odonto':
Tamanho (linhas, colunas): (10, 2)
Tipos de dados:
Mes     datetime64[ns]
Meta             int64
dtype: object
Primeiras linhas:
         Mes  Meta
0 2024-10-01    20
1 2024-09-01    20
2 2024-08-01    20
3 2024-07-01    20
4 2024-06-01    20


C:\Anaconda\lib\site-packages\google\cloud\bigquery\_pandas_helpers.py:489: FutureWarning: Loading pandas DataFrame into BigQuery will require pandas-gbq package version 0.26.1 or greater in the future. Tried to import pandas-gbq and got: No module named 'pandas_gbq'
  warnings.warn(


✅ Dados enviados com sucesso para: telemedicina-462920.seguros.meta_odonto
